In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt 

from IPython.display import display
plt.style.use('fivethirtyeight')
sns.set_style({'font.sans-serif':['simhei','Arial']})
%matplotlib inline


In [ ]:
%%markdown 

### 数据初探 
- 了解数据特征的缺失值、异常值以及大概的描述性统计


In [ ]:
lianjia_df = pd.read_csv("lianjia.csv")
display(lianjia_df.head())

In [ ]:
%%markdown 
- 初步了解数据共有`11`个特征变量，`price`是目标变量（Y），

In [ ]:
lianjia_df.info()

In [ ]:
lianjia_df.describe()

In [ ]:
%%markdown
**`以上描述性分析发现数据集具有以下特质`**
- 发现数据集共有23677 Elevator特征有明显缺失
- Size 特征有可能存在异常值（异常值对模型性能影响较大 需要特别关注）

In [ ]:
%%markdown
### 数据可视化分析

In [ ]:
df_copy = lianjia_df.copy()
df_copy['PerPrice'] = lianjia_df['Price']/lianjia_df['Size']
df_copy['PerPrice'].head()

In [ ]:
# Region 分析

df_house_count = df_copy.groupby('Region')['Price'].count().sort_values(ascending=False).to_frame().reset_index()
df_house_mean  = df_copy.groupby('Region')['PerPrice'].mean().sort_values(ascending=False).to_frame().reset_index()


In [ ]:
f,[ax1,ax2,ax3] = plt.subplots(3,1,figsize=(20,15))
sns.barplot(x = 'Region',y='PerPrice',palette='Blues_d',data=df_house_mean,ax=ax1)
ax1.set_title("北京各大区二手房每平米单价对比",fontsize=15)
ax1.set_xlabel('区域')
ax1.set_ylabel('每平米单价')


sns.barplot(x = 'Region',y='Price',palette='Greens_d',data=df_house_count,ax=ax2)
ax2.set_title("北京各大区二手房数量对比",fontsize=15)
ax2.set_xlabel('区域')
ax2.set_ylabel('数量')


sns.boxplot(x = 'Region',y='Price',data=df_copy,ax=ax3)
ax3.set_title("北京各大区二手房总价",fontsize=15)
ax3.set_xlabel('区域')
ax3.set_ylabel('房屋总价')
plt.show()

In [ ]:
%%markdown
可以观察到：
- **二手房房均价：** 西城区房价最高大约为11万，因为西城区在二环以里，并且也是热门学区房的聚集地，其次是东城区大约为10万/平米，然后是海淀区大约为8.5万每平米

- **二手房数量：** 单从数量统计上来看，目前海淀、朝阳、丰台为二手房市场较为火热区域，差不多接近3000套
- **二手房总价：** 通过箱线图发现，各区房价中位数集中在1000万以下，且离散值较高，说明房屋价格特征不是理想的正态分布

In [ ]:
# Size 分析
f,[ax1,ax2] = plt.subplots(1,2,figsize=(15,5))
sns.distplot(df_copy["Size"],bins=20,ax=ax1,color='r')
sns.kdeplot(df_copy["Size"],shade=True,ax=ax1)
sns.regplot(x='Size',y="Price",data=df_copy,ax=ax2)
plt.show()


In [ ]:
%%markdown
> 通过distplot 和 kdeplot绘制柱状图观察Size特征分布情况，属于长尾类型分布，这说明有很多面积很大且超出正常范围的二手房
> 通过regplot 绘制Size 和 Price之间散点图，发现Size特征基本和Price呈线性关系，符合基本常识，面积越大 价格越高。但是有明显异常点存在
>   - 面积不足10平米，但是价格超出10000万
>   - 面积超过1000平米，但是价格很低
> 需要进一步分析

In [ ]:
df_copy[df_copy["Size"]<10]

In [ ]:
df_copy[df_copy["Size"]>1000]

In [ ]:
%%markdown
> 检查发现，这组数据为别墅，出现异常原因是由于别墅构造比较特殊（无朝向、无电梯）应该是该类型数据获取时出现的问题，且该类型不在解决问题范围可以考虑移除
> 大于1000平米数据异常有可能属于商业用房，已租赁为主，故选择删除

In [ ]:
df_copy = df_copy[(df_copy["Layout"]!="叠拼别墅") & (df_copy["Size"] < 1000)]
# Size 分析
f,[ax1,ax2] = plt.subplots(1,2,figsize=(15,5))
sns.distplot(df_copy["Size"],bins=20,ax=ax1,color='r')
sns.kdeplot(df_copy["Size"],shade=True,ax=ax1)
sns.regplot(x='Size',y="Price",data=df_copy,ax=ax2)
plt.show()


In [ ]:
# Renovation 特征分析
df_copy["Renovation"].value_counts()

In [ ]:
lianjia_df["Renovation"].value_counts()

In [ ]:
%%markdown
### Elevator 特征分析
- **初探数据时，我们发现Elevator特征缺失严重，对于我们分析来说十分不利，假如关键性特征缺失对于模型来讲是灾难性的。故而一般有平均值/中位数填补法、直接移除或者根据其他特征预测等**
- **观察缺失值数量**

In [ ]:
misn =  len(df_copy.loc[(df_copy["Elevator"].isnull()),"Elevator"])
print ("Elevator 缺失值数量为：",str(misn))

In [ ]:
%%markdown

- **在实际生活中，电梯与房价有很强的相关性，所以我们对数据进行填充**
- **通过观察数据发现，"Elevator" 特征为字符串类型，不存在平均值和中位数，因此根据楼层Floor来判断是否有电梯，一般情况当楼层大于6时有电梯**

In [ ]:
df_copy["Elevator"] = df_copy.loc[(df_copy["Elevator"] == "有电梯")|(df_copy["Elevator"] == "无电梯"),"Elevator"]

In [ ]:
# 填充缺失值
df_copy.loc[(df_copy["Floor"] > 6)&(df_copy["Elevator"].isnull()),"Elevator"] = "有电梯"
df_copy.loc[(df_copy["Floor"] <= 6)&(df_copy["Elevator"].isnull()),"Elevator"] = "无电梯"

In [ ]:
f,[ax1,ax2] = plt.subplots(1,2,figsize = (20,10))

sns.countplot(df_copy["Elevator"],ax=ax1)
ax1.set_title("有无电梯数量对比",fontsize=20)
ax1.set_xlabel("是否有电梯")
ax1.set_ylabel("数量")

sns.barplot(x="Elevator",y="Price",data=df_copy,ax=ax2)
ax2.set_title("有无电梯房价对比",fontsize=20)
ax2.set_xlabel("是否有电梯")
ax2.set_ylabel("总价")
plt.show()

In [ ]:
%%markdown 
> 观察发现，有电梯二手房居多，符合目前市场对于资源的合理利用，相应的有电梯房价相对较高（售价包含电梯装修和后期维修等相关费用（公摊面积））


In [ ]:
%%markdown
### Year 特征分析


In [ ]:
grid = sns.FacetGrid(df_copy,row="Elevator",col="Renovation",palette='seismic',size=4)
grid.map(plt.scatter,"Year","Price")
grid.add_legend()

In [ ]:
%%markdown 
> 在 Elevator 和 Renovation 分类条件下，使用FacGrid分析Year特征 观察结果如下：

- 整个二手房房价趋势是随着时间增长的（也就是离购买时间越近房价越高）
- 2000以后建造的二手房房价明显高于2000年以前的房价
- 1980年以前几乎不存在有电梯的二手房，说明1980年前并未修建太多高楼层和普及电梯（符合当时的市场环境和发展情况）
- 1980年之前二手房，简装占据大部分市场，也说明当时环境下商品房市场情况

In [ ]:
%%markdown
### Floor 特征分析


In [ ]:
f,ax1 = plt.subplots(figsize=(20,5))

sns.countplot(x = "Floor",data=df_copy,ax=ax1)
ax1.set_title("房屋户型",fontsize=20)
ax1.set_xlabel("数量")
ax1.set_ylabel("户型")
plt.show()

In [ ]:
%%markdown
可以看到，6 层二手房数量最多，但是单独的楼层特征没有什么意义，因为每个小
区住房的总楼层数都不一样，我们需要知道楼层的相对意义。另外，楼层与文化也
有很重要联系，比如中国文化七上八下，七层可能受欢迎，房价也贵，而一般也不
会有 4 层或 18 层。当然，正常情况下中间楼层是比较受欢迎的，价格也高，底层
和顶层受欢迎度较低，价格也相对较低。所以楼层是一个非常复杂的特征，对房价
影响也比较大。

In [ ]:
%%markdown  
### 特征工程

特征工程包括很多内容，有特征清洗、预处理、监控等，而预处理根据单一特征或多特征又分很多种方法，如归一化、降维、特征选择、特征筛选等。处理数据的好坏对模型的性能影响很大

In [ ]:
# 只考虑 “室” 和 “厅” 将其他少数“房间” 和 “卫” 移除

df_copy = df_copy.loc[df_copy["Layout"].str.extract('^\d(.*?)\d.*?',expand=False) == '室']

# 提取“室” 和 “厅” 创建新特征
df_copy["Layout_room_num"] = df_copy["Layout"].str.extract('(^\d).*',expand=False).astype("int64")
df_copy["Layout_hall_num"] = df_copy["Layout"].str.extract('(^\d).*',expand=False).astype("int64")
# 按中位数对“Year” 特征进行分箱

df_copy["Year"] = pd.qcut(df_copy["Year"],8).astype("object")


In [ ]:
df_copy = df_copy.loc[(df_copy["Direction"] != 'no')&(df_copy["Direction"]!='nan')]



In [ ]:
df_copy["Layout_total_num"] = df_copy["Layout_room_num"] + df_copy["Layout_hall_num"]
df_copy["Size_room_ratio"] = df_copy["Size"] / df_copy["Layout_total_num"]
df_copy = df_copy.drop(["Layout","PerPrice","Garden"],axis=1)


In [ ]:
df_copy.head()

In [ ]:
%%markdown 
### 数据建模预测
-  使用 Cart 决策树的回归模型对二手房房价进行分析预测
-  使用交叉验证方法充分利用数据集进行训练，避免数据划分不均匀的影响。
-  使用 GridSearchCV 方法优化模型参数
-  使用 R2 评分方法对模型预测评分

In [ ]:

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
ohe = preprocessing.OneHotEncoder()  
train = pd.DataFrame()
object_col = ['Direction', 'District', 'Elevator', 'Region',
       'Renovation', 'Size', 'Year', 'Layout_room_num', 'Layout_hall_num',
       'Layout_total_num', 'Size_room_ratio']

In [ ]:
for col in object_col:
    le.fit(df_copy[col])
    df_copy[col] = le.transform(df_copy[col])
    ohe.fit(df_copy[col].values.reshape(-1, 1))
    ohecol = ohe.transform(df_copy[col].values.reshape(-1, 1)).toarray()
    ohecol = pd.DataFrame(ohecol[:,1:],index=None)#columns=col+le.classes_
    ohecol.columns = ohecol.columns.map(lambda x:str(x)+col)
    train = pd.concat([train, ohecol],axis=1, ignore_index=False)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
"""
将以上数据划分为训练集和测试集，训练集用于建立模型，测试集用于测试模型预
测准确率。使用 sklearn 的 model_selection 实现以上划分功能。
"""
X_train,X_test,y_train,y_test = train_test_split(np.array(train),np.array(df_copy["Price"]),test_size=0.2)

In [ ]:
# 创建模型
from  sklearn.model_selection import KFold
from  sklearn.tree import DecisionTreeRegressor
from  sklearn.metrics import make_scorer
from  sklearn.model_selection import GridSearchCV
from  sklearn.metrics import r2_score



In [ ]:
def fit_model(X,y):
    cross_vaildator = KFold(10,shuffle=True)
    regressor  = DecisionTreeRegressor()
    
    params = {'max_depth':[1,2,3,4,6,5,7,9,10]}
    scoring_fnc = make_scorer(performance_metric)
    # 基于X，y进行网格搜索
    grid = GridSearchCV(estimator=regressor,param_grid=params,scoring=scoring_fnc,cv = cross_vaildator)
    grid.fit(X,y)
    return grid.best_estimator_

def performance_metric(y_ture,y_pred):
    return r2_score(y_ture,y_pred)


In [ ]:
# # 分析模型
# vs.ModelLearning(X_train, y_trian)
# vs.ModelComplexity(features_train, prices_train)
optimal_reg1 = fit_model(X_train, y_train)
# 输出最优模型的 'max_depth' 参数
print("最理想模型的参数 'max_depth' 是 {} 。".format(optimal_reg1.get_params()['max_depth']))
# 由于决策树容易过拟合的问题，我们这里采取观察学习曲线的方法查看决策树深
# 度，并判断模型是否出现了过拟合现象。以下是观察到的学习曲线图形：
predicted_value = optimal_reg1.predict(X_test)
r2 = performance_metric(y_test, predicted_value)
print("最优模型在测试数据上 R^2 分数 {:,.2f}。".format(r2))

In [ ]:
from  sklearn.linear_model import  LogisticRegression
from  sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score
logist = LogisticRegression()
logist.fit(X_train,y_train.astype('int'))
y_pred = logist.predict(X_test)


In [127]:
r2 = performance_metric(y_test, y_pred)
print ("最优模型在测试数据上 R^2 分数 {:,.2f}。".format(r2))


最优模型在测试数据上 R^2 分数 0.48。
